In [70]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from ydc.tools import import_data, distances
from ydc.tools.supercats import add_supercats
from simplekml import Kml, Style

from colorsys import hsv_to_rgb

#helper for color conv, copies from activestate.com and slightly modfied
def get_color(hue, saturation, alpha):
    rgb_tuple = hsv_to_rgb(hue, saturation,1)
    """ convert an (R, G, B) tuple to #AARRGGBB """
    rgb_tuple = (alpha*255, rgb_tuple[0]*255, rgb_tuple[1]*255, rgb_tuple[2]*255)
    hexcolor = '%02x%02x%02x%02x' % rgb_tuple
    return hexcolor

def prepare_kml(cat_dict, kml=None):
    # Prepare KML
    if kml is None:
        kml = Kml()

    # Create folder for businesses
    main_fol = kml.newfolder(name="business_densities")

    # Create subfolder for every category and save them in dictionary
    folders = {key: main_fol.newfolder(name=cat_dict[key]['name']) for key in cat_dict}

    # More subfolders for subcats
    subfolders = {}
    for superkey in folders:
        subcats = cat_dict[superkey]['sub_categories']
        subfolders[superkey] = {key: folders[superkey].newfolder(name=subcats[key]['name']) for key in subcats}
    
        # Add special for 'all'
        subfolders[superkey]['all'] = folders[superkey].newfolder(name='All')
    
    """
    #Create color for every category and also save them in dict
    styles = {}
    for superkey in cat_dict:
        subcats = cat_dict[superkey]['sub_categories']
        # Create dict of dicts with entries for every subcategory
        styles[superkey] = {key: Style() for key in subcats}
        for key in subcats:
            styles[superkey][key].labelstyle.scale = 0
            hue = superkey/len(cat_dict.keys())  # Around the circle
            sat = 0.5 + key/(2 * len(subcat.keys()))  # Avoid the middle so not everything gets white
            styles[superkey][key].iconstyle.icon.href = (
                'http://thydzik.com/thydzikGoogleMap/markerlink.php?color=%s' % get_color(hue, sat)
            )

    #Special case for uncategories (those are -1 and will be black)
    styles[-1] = {-1: Style()}
    styles[-1][-1].labelstyle.scale = 0
    styles[-1][-1].iconstyle.icon.href = 'http://thydzik.com/thydzikGoogleMap/markerlink.php?color=000000'
    """
    
    return kml, subfolders

In [3]:
df = import_data.import_businesses()
cells = distances.CellCollection(15, df)

In [4]:
cell_copy = cells.to_dict()

In [9]:
(supercats_frame, box) = add_supercats(import_data.import_businesses())  # cats love boxes!

In [ ]:
cat_cells = {}
for col_name, col in cell_copy.items():
    for row_name, cell in col.items():
        super_cats = {}
        sub_cats = {}
        for business in cell:
            # Count super and sub categories for businesses in this cell
            super_idx, sub_idx = supercats_frame.iloc[business['index']][['super_category','sub_category']]
            
            # This counts super only, ignores which sub cat
            super_cats[super_idx] = super_cats.get(super_idx, 0) + 1
            
            # More specific, sorted by super and sub cat
            if super_idx not in sub_cats.keys():
                sub_cats[super_idx] = {}
            sub_cats[super_idx][sub_idx] = sub_cats[super_idx].get(sub_idx, 0) + 1
            
            # store into new dict by cell
            if col_name not in cat_cells.keys():
                cat_cells[col_name] = {}
            cat_cells[col_name][row_name] = {'super': super_cats, 'sub': sub_cats}
 

In [49]:
# Find maxima
suplist = []
for col in cat_cells:
    for row in cat_cells[col]:
        for superkey in cat_cells[col][row]['super']:
            suplist.append(cat_cells[col][row]['super'][superkey])
            
max_density = max(suplist)

# Norm everything
for col in cat_cells:
    for row in cat_cells[col]:
        for superkey in cat_cells[col][row]['super']:
            cat_cells[col][row]['super'][superkey] /= max_density
            
            for subkey in cat_cells[col][row]['sub'][superkey]:
                cat_cells[col][row]['sub'][superkey][subkey] /= max_density

In [73]:
city= 'Edinburgh'

businesses = import_data.import_businesses()
businesses_in_city = businesses[businesses['city']==city]

def make_polygon(folder, lowerleft, upperright, color):
    pol = folder.newpolygon(name='A Polygon')
    pol.extrude = 1
    coords = [lowerleft, (lowerleft[0], upperright[1]), upperright, (upperright[0], lowerleft[1]), lowerleft]
    pol.outerboundaryis = coords
    pol.style.polystyle.color = color
    

def make_polygons(folders, lowerleft, upperright, cell):
    for superkey in cell['super']:
        sup_tot = len(cell['super'].keys())
        
        hue = superkey/sup_tot  # Around the circle
        sat = 1  # on the outside
        alpha = cell['super'][superkey] 
        
        fol = folders[superkey]['all']
        make_polygon(fol, lowerleft, upperright, get_color(hue, sat, alpha))
    
        for subkey in cell['sub'][superkey]:  # all sub cats for this super cat
            sub_tot = len(cell['sub'][superkey].keys())
            
            hue = superkey/sup_tot  # Around the circle
            sat = 1  # 0.5 + subkey/(2 * sub_tot)  # Avoid the middle so not everything gets white            
            alpha = cell['sub'][superkey][subkey]
            
            fol = folders[superkey][subkey]
            make_polygon(fol, lowerleft, upperright, get_color(hue, sat, alpha))
    
# Prepare KML
kml, folders = prepare_kml(box)


# Get boundaries for City
min_x = cells.get_cell(businesses.iloc[businesses_in_city['longitude'].idxmin()])[0]
max_x = cells.get_cell(businesses.iloc[businesses_in_city['longitude'].idxmax()])[0]
min_y = cells.get_cell(businesses.iloc[businesses_in_city['latitude'].idxmin()])[1]
max_y = cells.get_cell(businesses.iloc[businesses_in_city['latitude'].idxmax()])[1]
    
(longitudes, latitudes) = cells.get_borders()

for x in range(min_x, max_x):
    for y in range(min_y, max_y):
        if x in cat_cells and y in cat_cells[x]:
            make_polygons(folders, (longitudes[x], latitudes[y]), (longitudes[x+1], latitudes[y+1]), cat_cells[x][y])
            
kml.save("%s.kml" % city)

In [56]:
folders['super']

{-1: <simplekml.featgeom.Folder at 0x23b3ee10>,
 0: <simplekml.featgeom.Folder at 0x6603a278>,
 1: <simplekml.featgeom.Folder at 0x6603a2e8>,
 2: <simplekml.featgeom.Folder at 0x6603a358>,
 3: <simplekml.featgeom.Folder at 0x6603a5c0>,
 4: <simplekml.featgeom.Folder at 0x23b3e8d0>,
 5: <simplekml.featgeom.Folder at 0x23b3ea20>,
 6: <simplekml.featgeom.Folder at 0x23b3ebe0>,
 7: <simplekml.featgeom.Folder at 0x23b3eb38>,
 8: <simplekml.featgeom.Folder at 0x23b3ecc0>}

In [63]:
import simplekml

In [64]:
simplekml.Color.red

'ff0000ff'

In [69]:
get_color(0.7,1,0)

'3200ffff'